# Importing the Libraries


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


###### Data retrieved from City of Surrey Open Data 
https://data.surrey.ca/dataset/restaurants

In [2]:
df = pd.read_csv(r'D:\restaurants.csv')

In [3]:
df.head()

,TRACKINGNUMBER,NAME,PHYSICALADDRESS,PHYSICALCITY,FACTYPE,LATITUDE,LONGITUDE
0,SWOD-APSP3X,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,SDFO-88ESVK,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,NDAA-8RNNVR,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,SYOG-5M5942,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,GEDS-6KHW57,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [6]:
pwd

'C:\\Users\\srira'

# Data Wrangling

In [4]:
df.drop('TRACKINGNUMBER',axis=1,inplace = True)

In [5]:
df.head()

,NAME,PHYSICALADDRESS,PHYSICALCITY,FACTYPE,LATITUDE,LONGITUDE
0,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [6]:
df.rename(columns={'PHYSICALADDRESS':'ADDRESS','PHYSICALCITY':'CITY'},inplace = True)

In [7]:
df.head()

,NAME,ADDRESS,CITY,FACTYPE,LATITUDE,LONGITUDE
0,104 Sushi & Co.,10422 168 St,Surrey,Restaurant,49.192059,-122.756256
1,5 Star Catering,5640 188 St,Surrey,Restaurant,49.105449,-122.701315
2,555 Pizza Ltd,9192 120 St,Surrey,Restaurant,49.170274,-122.889988
3,7-Eleven #26365,7986 120 St,Surrey,Restaurant,49.148189,-122.890221
4,7-Eleven #26517,17638 60 Ave,Surrey,Restaurant,49.111322,-122.734271


In [8]:
df.drop('FACTYPE',axis=1,inplace = True)

In [36]:
df = df.head(500)
df.shape

(500, 5)

In [37]:
address = 'Surrey, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\srira\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 49.1913033, -122.8491439.


In [11]:
map_surrey = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(df['LATITUDE'], df['LONGITUDE'], df['NAME']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_surrey)  
    
map_surrey

##### Getting the foursquare credentials

In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


# Exploratory Data Analysis

##### Exploring the first restaurant and the nearby venues

In [39]:
# df.loc[0,'ADDRESS']
latitude = df.loc[1,'LATITUDE']
longitude = df.loc[1,'LONGITUDE']

##### Setting the foursquare request URL

In [40]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ONAZS2PTG1GXEGVC5IFRCK4H3E5LLKODWWCXIBCQGI1VZBHC&client_secret=0XJQQBLBWNGYGWKLTNHOWDBWAAJMY53XOXOVF0WX3WRNCQ0R&v=20180605&ll=49.10544931,-122.70131471&radius=500&limit=100'

In [41]:
results = requests.get(url).json()
# results

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

nearby_venues1.head()

,name,categories,lat,lng
0,The Clayton Public House,Pub,49.104655,-122.701060
1,Sunrise Golf Centre,Golf Course,49.105024,-122.701071
2,Mavericks Taphouse And Grill,Pub,49.104966,-122.701028


##### The above cell shows that there are three popular venues near the first restaurant ' 104 Sushi and Co' . There are two pubs and a golf course. Probably not a good idea to open a restaurant near noisy unpleasant pubs

##### Now lets get the nearby venues for all the restaurants from our Dataset

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['RESTAURANT NAME', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
surrey_venues = getNearbyVenues(names=df['NAME'],
                                   latitudes=df['LATITUDE'],
                                   longitudes=df['LONGITUDE']
                                  )

104 Sushi & Co.
5 Star Catering
555 Pizza Ltd
7-Eleven #26365
7-Eleven #26517
7-Eleven #34339
7-Eleven #34501
7-Eleven #34633
7-Eleven #34907
7-Eleven #35770
7-Eleven #36022H
7-Eleven #37876H
7-Eleven #38206
7-Eleven (Canada Store #37206H)
7-Eleven (Canada Store #37865H)
7-Eleven (Canada Store #37866H)
7-Eleven Canada Store #36676
7-Eleven Store # 37868
7-ElevenStore and Gas Bar (Canada Store #37875H)
North Surrey Secondary School Cafeteria
Northview Golf & Country Club (Canal Kiosk)
Northview Golf & Country Club (Kitchen/Duffey's Sports Grill)
Northview Golf & Country Club (Palmer Dining Room)
Northview Golf & Country Club (Ridge Kiosk)
Notti Biscotti
Nutri-Dough Pizza
O My Tea Cafe
Occasions at the Pond
Ocean Park Donair
Ocean Park Pizza
Ocean Park Pizza & Village Pub
Ocean Park Pizza (72nd Ave)
Oh So Sweet Sugar Boutique
Ok Hee Bae Food Co
Okonomiyaki Pon
Old Surrey Restaurant
Omega Pizza & Wings
Omega Pizza & Wings (152nd St)
On Yuen Restaurant
One More Thai Restaurant
One Up Cafe


In [49]:
surrey_venues.head(10)

,RESTAURANT NAME,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,104 Sushi & Co.,49.192059,-122.756256,104 Sushi & Co,49.192245,-122.756318,Sushi Restaurant
1,104 Sushi & Co.,49.192059,-122.756256,Flying Cup,49.191845,-122.756420,Coffee Shop
2,104 Sushi & Co.,49.192059,-122.756256,Fraser Glen Pizza,49.191401,-122.756039,Pizza Place
3,104 Sushi & Co.,49.192059,-122.756256,777 Bubble Tea,49.191919,-122.756211,Tea Room
4,5 Star Catering,49.105449,-122.701315,The Clayton Public House,49.104655,-122.701060,Pub
5,5 Star Catering,49.105449,-122.701315,Sunrise Golf Centre,49.105024,-122.701071,Golf Course
6,5 Star Catering,49.105449,-122.701315,Mavericks Taphouse And Grill,49.104966,-122.701028,Pub
7,555 Pizza Ltd,49.170274,-122.889988,Thai By Thai,49.169821,-122.889959,Thai Restaurant
8,7-Eleven #26365,49.148189,-122.890221,The Keg Steakhouse + Bar,49.147487,-122.890022,Steakhouse
9,7-Eleven #26365,49.148189,-122.890221,Cactus Club Cafe,49.146924,-122.890932,American Restaurant


##### Here is where it gets interesting. For a restaurant, the nearby venues retrieved from Foursquare shoul return the same restaurant as a spot as well. Well, only if the restaurant is popular or  preferred. As you can see above,  for some restaurants like the '5 Star Catering', the nearby venues doesnt include them in the results. This could be because they might be just a 'take away' restaurant with no dine in facilities. In such cases, its a potential good spot without having to worry about competition.

In [47]:
surrey_venues.shape

(2613, 7)

#####  How many popular venues are around each restaurant?

In [48]:
surrey_venues.groupby('RESTAURANT NAME').count()

,Restaurant Latitude,Restaurant Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
RESTAURANT NAME,,,,,,
104 Sushi & Co.,4,4,4,4,4,4
5 Star Catering,3,3,3,3,3,3
555 Pizza Ltd,1,1,1,1,1,1
7-Eleven #26365,9,9,9,9,9,9
7-Eleven #26517,3,3,3,3,3,3
7-Eleven #34339,1,1,1,1,1,1
7-Eleven #34501,3,3,3,3,3,3
7-Eleven #34633,12,12,12,12,12,12
7-Eleven #34907,4,4,4,4,4,4


In [50]:
print('There are {} uniques categories.'.format(len(surrey_venues['Venue Category'].unique())))

There are 163 uniques categories.


##### One hot Encoding the Venues

In [51]:
surrey_onehot = pd.get_dummies(surrey_venues[['Venue Category']], prefix="", prefix_sep="")

# # add neighborhood column back to dataframe
surrey_onehot['RESTAURANT NAME'] = surrey_venues['RESTAURANT NAME'] 

# # move neighborhood column to the first column
fixed_columns = [surrey_onehot.columns[-1]] + list(surrey_onehot.columns[:-1])
surrey_onehot = surrey_onehot[fixed_columns]

surrey_onehot.head(10)

,RESTAURANT NAME,Afghan Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Campground,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Rec Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Deli / Bodega,Department Store,Design Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Duty-free Shop,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Irish Pub,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Music Store,National Park,Neighborhood,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,104 Sushi & Co.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5 Star Catering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

##### Grouping the results by the restaurants

In [52]:
surrey_grouped = surrey_onehot.groupby('RESTAURANT NAME').mean().reset_index()

In [53]:
surrey_grouped.head()

,RESTAURANT NAME,Afghan Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Big Box Store,Bistro,Board Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Campground,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Rec Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Deli / Bodega,Department Store,Design Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Duty-free Shop,Electronics Store,Event Service,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Irish Pub,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Music Store,National Park,Neighborhood,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,104 Sushi & Co.,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5 Star Catering,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,555 Pizza Ltd,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

In [54]:
surrey_grouped.shape

(461, 164)

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Sorting the venues 

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['RESTAURANT NAME']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
surrey_venues_sorted = pd.DataFrame(columns=columns)
surrey_venues_sorted['RESTAURANT NAME'] = surrey_grouped['RESTAURANT NAME']

for ind in np.arange(surrey_grouped.shape[0]):
    surrey_venues_sorted.iloc[ind, 1:] = return_most_common_venues(surrey_grouped.iloc[ind, :], num_top_venues)

surrey_venues_sorted.head()

,RESTAURANT NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,104 Sushi & Co.,Tea Room,Pizza Place,Coffee Shop,Sushi Restaurant,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,5 Star Catering,Pub,Golf Course,Yoga Studio,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,555 Pizza Ltd,Thai Restaurant,Event Service,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
3,7-Eleven #26365,Restaurant,American Restaurant,Other Repair Shop,Convenience Store,Greek Restaurant,Indian Restaurant,Doctor's Office,Coffee Shop,Steakhouse,Yoga Studio
4,7-Eleven #26517,Flea Market,Convenience Store,Gas Station,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


# Model Development

##### Using K Means Clustering to cluster an segment the community based on the restaurants and the popular spots returned by foursquare

In [60]:
kclusters = 5

surrey_grouped_clustering = surrey_grouped.drop('RESTAURANT NAME', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(surrey_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([0, 4, 4, 4, 4, 4, 0, 4, 3, 4, 0, 3, 3, 0, 3, 3, 4, 4, 3, 4, 3, 3,
       3, 3, 3, 4, 4, 3, 0, 4])

In [65]:
surrey_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



# surrey_merged.head()



In [64]:
surrey_venues_sorted.head()

,Cluster Labels,RESTAURANT NAME,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,104 Sushi & Co.,Tea Room,Pizza Place,Coffee Shop,Sushi Restaurant,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,4,5 Star Catering,Pub,Golf Course,Yoga Studio,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,4,555 Pizza Ltd,Thai Restaurant,Event Service,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
3,4,7-Eleven #26365,Restaurant,American Restaurant,Other Repair Shop,Convenience Store,Greek Restaurant,Indian Restaurant,Doctor's Office,Coffee Shop,Steakhouse,Yoga Studio
4,4,7-Eleven #26517,Flea Market,Convenience Store,Gas Station,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


### Cluster 1

##### This cluster is a cluster of yoga studios, stores, pizza and sandwich places, juice bars but mostly inclues the fish market nearby. What an useful insight !

In [66]:
surrey_venues_sorted.loc[surrey_venues_sorted['Cluster Labels'] == 0, surrey_venues_sorted.columns[[1] + list(range(5, surrey_venues_sorted.shape[1]))]]

,RESTAURANT NAME,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,104 Sushi & Co.,Sushi Restaurant,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
6,7-Eleven #34501,Yoga Studio,Fabric Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
10,7-Eleven #36022H,Sandwich Place,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
13,7-Eleven (Canada Store #37206H),Chinese Restaurant,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
28,A&W #0735,Pizza Place,Greek Restaurant,Breakfast Spot,Supermarket,Restaurant,Event Service,Filipino Restaurant
39,Afghan Bakery and Fast Food,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
53,Ambros Subway,Sandwich Place,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
54,An Pho Vietnamese Restaurant,Japanese Restaurant,Yoga Studio,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
57,Aomori Sushi Japanese Restaurant,Grocery Store,Indian Restaurant,Sandwich Place,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market
70,BBQ Nation,Portuguese Restaurant,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [67]:
surrey_venues_sorted.loc[surrey_venues_sorted['Cluster Labels'] == 1, surrey_venues_sorted.columns[[1] + list(range(5, surrey_venues_sorted.shape[1]))]]

,RESTAURANT NAME,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
137,Northview Golf & Country Club (Canal Kiosk),Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
138,Northview Golf & Country Club (Kitchen/Duffey'...,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
139,Northview Golf & Country Club (Palmer Dining R...,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
140,Northview Golf & Country Club (Ridge Kiosk),Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
196,Peace Portal Golf Course Lounge,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
197,Peace Portal Golf Course Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop


In [68]:
surrey_venues_sorted.loc[surrey_venues_sorted['Cluster Labels'] == 2, surrey_venues_sorted.columns[[1] + list(range(5, surrey_venues_sorted.shape[1]))]]

,RESTAURANT NAME,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Afghan Chopan,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
42,Aggarwal Sweets,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
47,Akash Chaat House & Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
58,Apna Chaat House,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
81,Barney's Pizza & Chicken,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
83,Basant Sweet,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
117,Bollywood Banquet Hall,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
136,North Surrey Secondary School Cafeteria,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
149,Ocean Park Pizza (72nd Ave),Yoga Studio,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
193,Payal Chaat House & Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [69]:
surrey_venues_sorted.loc[surrey_venues_sorted['Cluster Labels'] == 4, surrey_venues_sorted.columns[[1] + list(range(5, surrey_venues_sorted.shape[1]))]]

,RESTAURANT NAME,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,5 Star Catering,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,555 Pizza Ltd,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
3,7-Eleven #26365,Convenience Store,Greek Restaurant,Indian Restaurant,Doctor's Office,Coffee Shop,Steakhouse,Yoga Studio
4,7-Eleven #26517,Event Service,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
5,7-Eleven #34339,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
7,7-Eleven #34633,Chinese Restaurant,Sushi Restaurant,Bank,Coffee Shop,Gas Station,Convenience Store,Discount Store
9,7-Eleven #35770,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
16,7-Eleven Canada Store #36676,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Fabric Shop
17,7-Eleven Store # 37868,Restaurant,Gas Station,Gift Shop,Fast Food Restaurant,Grocery Store,Convenience Store,Gym / Fitness Center
19,777 Pizza & Donair,Rock Climbing Spot,Yoga Studio,Event Service,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


##### We can find the perfect spot for the new restaurant based on the above clusters